## Datastrukturer

Bygger på numpy arrays og arver mye funksjonalitet derfra. Det er to sentrale objekt: Series og DataFrame

I tillegg til den implisitte numeriske indexen til arrays har den også et eksplisitt indexobjekt som mapper til element i Series eller til Series i DataFrame. Koblingen mellom indeks og verdi gjør at vi kan kombinere data fra ulike kilder, håndtere missing data og generelt ikke er avhengig av å ha helt konforme data (samme størrelse, samme rekkefølge) for å gjøre regneoperasjoner.

### Series

Series er crosseover mellom array og dict. Består av to arrays: én index og én med homogene data. Vanlige ndarrays har implisitt index med posisjon, men her er det eksplisitt og kan ha ulike labels. Kan også tenke på Series som en dict som mapper index-verdier til verdi i array. Følgelig er det en del metoder/syntax som tilsvarer dict. Kan få ut hver av arraysene med:
1. a.values (ndarray)
2. a.index (index-objekt, finnes litt ulike typer, har diverse metoder, noe mengdegreier)

### DataFrame

DataFrame er tabulær datastruktur med både rekke-index og kolonne-index. I utgangspunktet er det ganske symmetrisk, men av konvensjon angir rekke-index obseravjon og kolonne-index er variabel (dimensjon/egenskap) ved observasjonen. Kan betrakte det som en dict som mapper label til Series, der labels er enten fra rekke- eller kolonneindex.

### Index

Det er en fullverdi datastruktur/objekt i seg selv. Har en del metoder og sånn. Vet ikke om jeg må jobbe så mye direkte med indexen; kan gjøre operasjon på dataframe og så håndterer de index internt.


Merk at når vi setter en ny kolonne som indeks mister vi den eksisterende index-kolonnnen. Hvis vi vil beholde må vi lagre kopi av kolonne og deretter putte den inn i nye dataframe. Kan få tak i kolnnen ved å bruke df.index() og deretter dytte den inn i df ved å assigne kolonnen med df["navn"] = df.index()


#### Indeksing

Kan legge til verdier på subset av dataframe

```python
df.loc['a'] = arr
df.loc['a'] = df_sub # fungerer ikke selvom matchene index, vet ikke hvorfor ...
```

#### kolonner

Kan merke at kolonnene egentlig bare er index og at det i utgangspuntket er ganske symmetrisk.

Vil ha deskriptive kolonnenavn. Bruker
- df.rename(columns=mapper), der mapper er dictionary med {'gammel':'ny',:}

Tror min konvensjon er å ha navn i snake_case, kan bruke

Kan bruke vanlig list comprehension til å finne subset av kolonner som oppfyller kriterie,

```python
cols_subset = [col for col in df.columns if 'string' in col]
```

#### Multi index

Index med flere level. Ytterste er level=0, og så teller vi oppover 

##### Konstruere multiindex

Kan eksplisitt konstruere multiindex objekt på flere måter

```python
arrays = [np.array(["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"]),
          np.array(["one", "two", "one", "two", "one", "two", "one", "two"])]

# Tre ekvivalente indekser:
pd.MultiIndex.from_tuples(zip(arrays[0],arrays[1])) # trenger list of tuples
pd.MultiIndex.from_arrays(arrays) # trenger list of arrays
pd.MultiIndex.from_frame(pd.DataFrame(arrays).T) # organisere arrays i dataframe før vi lage index

pd.MultiIndex.from_product(iterables) # alle unike kombinasjoner av verdier i ulike arrays
```

Kan også konstruere MultiIndex i constructor for DataFrame, må da være list of arrays
```python
df = pd.DataFrame(index=arrays, data=np.random.randn(len(index)), columns=['tall'])
df.set_index('tall', append=True) # legge til kolonne på eksisterende index

# Hvis vi ikke har eksisterne index kan vi lage nye multiindex med:
df.set_index(["kol1","kol2"])
```

Hvis vi har lyst til å legge ny array (eller eventuelt konstant verdi) som øvereste level i multiindex bør vi transformere index til dataframe så den  blir enklere å manipulere
```python
temp = df.index.to_frame()
temp.insert(0, 'name', value)
df.index = pd.MultiIndex.from_frame(temp)
```

##### Slice multiindex

Subsetting av observasjoner avhengig av verdier i multiindex kan bli ganske komplisert.

```python

df.loc['a'] # alle rekker der index i level 0 == 'a'
df.loc[('a', 'b')] # alle rekker der level 0 == 'a' og level 1 == 'b'
```

Må se mer på dette senere

#### Sortering

Kan sortere entent etter index eller verdi langs gitt kolonne,
1. df.sort_index()
2. df.sort_values(['index','kol']), kan sende inn både navn på index og kolonne...

## Lage dataframes

### Fra fil

Spesifiserer hvordan pd.readcsv() skal laste inn data

1. usecols = [..] for å spesifisere subset av kolonner vi vil laste inn
2. names = [..] hvis kolonnene ikke har navn fra før så kan vi spesifisere de.
3. Bestemme hvilken kolonne i datasett som skal fungere som index, index_col = <tall>. 
4. Dersom det er rekke vi ikke vil laste inn, skiprows = <tall>, skipper de n øverste.
5. parse_dates = True ---> finner kolonne med date, gjør den til index, får den i datetime.
    
Kan også laste inn fra andre filtyper enn .csv, for eksempel .json. Tar da orient keyword med verdi "split", "records" eller "columns"
    
pd.read_json(orient=)

```python
pd.read_csv(filename,
            sep=',', # kan bruke andre seps, tror tab er '\t'
            delimiter=None,
            header='infer', # må spesifisere header=0 for å endre navn..
            names=None,
            index_col=None, # hvis første kol skal være index bruker vi =0
            usecols=None,
            parse_dates, # ?
            
            ...)
```

### Fra datastrukturer

Kan ha lyst til å lage tabell som mapper index til verdi. F.eks. tabell med som mapper variabelnavn til reg koef. Har to lister. Et alternativ er pd.Series(data=x,index=y). Annet alternativ er å først konstruere dict:

pd.Series(dict(zip(y,x))

### Lagre dataframe

Kan lagre dataframes som består av tall og strenger som tekst i csv format med `to_csv`

Hvis det inneholder andre objekter, så må vi lagre til `to_pickle`. Generelt vil pickle lagre fullstendig informasjon om dataframe; alt fra datatyper, form av indeks, ... alt. Når vi lagrer til csv så er det bare en lang string og vi må tolke denne stringen når vi laster dataframe, så da starter vi gjerne fra scratch. Pickle er bedre.

## Beskrive data

Før jeg kan gjøre noe analyse må jeg først undersøke tabellen med tall jeg har fått utdelt. Må få oversikt over:
1. Hvilke egensakper (kolonner) og hva de måler
2. Hva slags måleenhet de er i og konvertere til riktig datatype. I algebraen under the hood er alt tall, men representer som factor for å få bedre representasjon og utnytte funksjonalitet i plotte-libraries og lignende. Vil unngå å jobbe eksplisitt med dummies før det er nødvendig.
3. Få noe oversikt over univariate fordelinger og eventuelt parvis bivariate korrelasjoner. Se at tall er rimelig, blir kjent med data, oppdage numerisk kodete missing values, se etter feilkodinger
4. Få oversikt over manglende verdier, vurdere strategi for imputation og eventuelt bias dersom vi ser på delutvalg med fullstendig obervasjon.

Kan bruke .describe() --- output avhenger av datatype

Kan bruke .info() til å få datatype og observere om det er missing values

Kan bruke .ndtype() for å finne datatypene

Hvis vi har en series kan vi bruke .value_count() for å få antallet observasjoner med gitt verdi langs den dimensjonen.

Kan bruke .unique() for å få liste av unike verdier og .unique() for antallet unike

### Datatyper

Hva slags type data vi har påvirker gyldige operasjoner og analyse av data. Derfor viktig å kategorisere. Hver kolonne er homogen.

1. Ulike numeriske datatyper (int,float,..)
2. category
3. object

Kan lage ordered categorical med pd.Categorical(df.col, ordered=True, categories = [..]), der categories er optional og kun dersom jeg trenger gitt rekkefølge (fra lav til høy).

Kan endre datatype til flere kolonner ved : df[liste_av_kolonner] = df[liste_av_kolonner].astype('dtype') 

```python
cat_cols = [col for col in df.columns if df[col].dtype == 'object']
df[cat_cols] = df[cat_cols].astype('category')
```

Kan lage kategorier fra numerisk verdi med dictionary som mapper verdi til kategori og col.map(table). Det forutsetter at mapping er exhaustive siden resten blir nans. Kan eventuelt bruke .fillna(df['col1']) etterpå for å beholde andre verdier.

### Beskrive kolonner

Bruker value_counts(normalize=).sort_index() # normalize hvis jeg vil ha som prosent

gir oversikt over antall unike verdier, hvor tyngden av data ligger og om det er verdier som ikke gir mening.

## Missing values

Vil representere med NaN. Kan lage med np.nan. Dette er en float, slik at kolonner med missing values blir recast til floats. Det er litt teknisk hvordan det blir representert under the hood, får eventuelt se på det senere. I R er det na og null, i pandas bare NaN (finnes en mer moderne datatype na som de har lagt til).

Missing values propapegerer når vi gjør operasjoner på dataframe. Operasjon er elementvis og alt som bruker NaN resultererr i NaN. Dette gjelder også får aggregeringsfunksjoner.

### Filtrering

Første jeg må gjøre er å sjekke antallet missing values i ulike kolonner. Husk at missing values kan være kodet på ulike måter i ulike datasett. En mulighet er å bruke df['col'].unique() til å se på verdier og se om noen ikke gir mening. Deretter vil jeg omkode de til NaN, f.eks. ved df.replace(value,np.nan)

- df.isna() gir boolean mask over hele dataframe
- df.isna().any() angir om hvorvidt det er nans i hver av kolonner
- df.isna().sum() gir antallet nans i hver kolonne

Når jeg har fått oversikt kan jeg vurdere å filtrere de ut. Bruker da df.dropna() med opsjoner
- axis= .. droppe kolonne eller rekke avhengig om jeg finner nans der
- how='any','all'
- tresh= antall nans før jeg dropper

### Imputation

En alternativ fremgangsmåte er å fylle inn verdier. Algoritmer i sklearn kan ikke håndtere nans og det er synd å kaste bort data bare fordi noen observasjoner er litt mangefulle. Finnes ulike strategier for dette som jeg kan se på senere
1. Bruke gjennomsnitt i kolonnen
2. Predikere verdi ut fra andre dimensjoner vi observere
3. Bruke verdi fra naboer

## Data cleaning

Arrays i dataframe er homogone. Dersom det er noen symboler i kolonne som i utgangspunktet skal være numerisk vil pandas tolke det som string. Må fjerne symbolene før vi kan konvertere.
```python
def to_numeric(s):
    return int(''.join(ch for ch in s if ch.isnumeric()))
```

## Split-apply-combine workflow

Vi har et stort datasett. Det er mange variabler og mange kolonner. Hvordan skal vi få ut noen innsikter fra all denne informasjonen?

Kan vært veldig nyttig å dele det inn i mindre deler etter felles verdi langs kolonne. Kan tenke at det er observasjoner som "hører sammen". Deretter annvender vi noen aggregeringsfunksjoner som gir oss slags sammendragsmål for tallverdiene i hver del. Deretter kan vi kombinere det sammen i nytt datasett som mapper nøkkel (felles verdi) til verdi av aggregeringsfunksjon.

Okay, la oss se på dette i praksis:

gbA = df.groupby('A') gir oss et groupby objekt. Det er poeng at hver gruppe er en dataframe. Kan få ut disse dataframene med gbA.get_group(verdi av A). Kan gruppere med verdier langs flere kolonner; dette gir og PxK grupper, der P er verdi langs første og K er verdi langs andre. I den kombinere dataframen vil har multiindex. Kan også observere at hver observasjon er i én gruppe.

Kan anvende aggregeringsmetoder direkte på gbA. Kan også definere custom aggregeringfunksjon og anvende de med .agg metoden

gbA.agg(lambda x: (x > 0).count()) gir antaller positive observasjon. Kan være greit å gi funksjonene navn så blir litt mer ryddig.

For å øke fleksibiliteten til vår oppdeling av datasett kan vi lage et Grouper objekt. Stor fordel her Grouperen vår kan tolke datetime objekter. Hvis key=series med datetime så kan vi slenge in freq="W" og gruppere observasjoner for hver uke

eksempel: gb = df.groupby(pd.Grouper(key='Date', freq ='W')) der Date er kolonne med datetime objekt.

### groupby

Det er en måte å konstruere separate dataframes filtrert på verdi av kolonne vi grupper etter. Kan få tak i de interne dataframene med
```python
for idx, df in df.groupby('col')[andre_cols]:
    print(idx) # verdi av col
    print(df) # dataframe filtrert på col == idx
```    
i praksis så bruker vi groupby ofte til å lage en ny dataframe med sammendragsmål på betinget fordeling for hver verdi av 'col'.

er noe distinksjon mellom dataframe groupby og series groupby. får dataframe groupby hvis jeg spesifiser as_index=False. hmm. poeng at det kjører metode på entent dataframe eller series under the hood og er ikke fullstendig overlapp der...

vil i alle tilfeller at output er dataframe i stedet for series. kan oppnå dette med .reset_index(). Viktig at series har navn da slik at jeg får navn i kolonne av ny dataframe.

Hvis det er jeg grupperer etter id på stasjon og det er samme navn på alle stasjon med samme id.. og jeg vil ha ut kolonne med id og navn. Så kan jeg bruke .first() i stedet for å aggregere

```python
cols = ['start_station_name', 'start_station_latitude', 'start_station_longitude']
stations = df.groupby('start_station_id')[cols].first()
```

#### Multi-level groupby

Kan ha lyst til å gruppere på to eller flere kolonner samtidig. En mulig fremgangsmåte ville vært å ta groupby på hver group av første kolonne og så fått alle verdiene inn en ny dataframe, eg
```python
df_out = pd.DataFrame(index=some_index)
gb = df.groupby('first_column')
for idx, group in gb:
    group.loc[idx] = group.groupby('second_column')['some_column'].mean()
```
eller noe sånt. Dette er veldig rotete. Heldigvis gjør pandas dette enkelt ved å sende inn flere kolonner i groupby
```python
df_out = df.groupby(['first_column', 'second_column'])['some_column'].mean()
```
Merk at df_out bare vil ha index for de kombinasjoner av verdier i kolonnene vi grupperer på som vi faktisk observerer i data. I mange tilfeller kan det være greit å ha alle kombinasjonene og padde med NaNs for de vi ikke observerer. Litt usikker på hvordan jeg skal gjøre dette... må resette/konstruere denne indeksen på en måte.

### resample

Brukes for å aggrere tidsserie observasjoner innenfor tidsintervall.

```python
df.resample(rule='H', # hourly, daily, weekly, monthly... spesifisere intervall
            on='started_at' # kolonne med tidsdimensjon (hvis ikke indeks)
            )
            


### grouper

Bruker inni groupby for mer kontroll.

Eksempel: gruppere observasjoner med timedelta innefor 5 minutters intervall
```python
duration_agg = df.groupby(pd.Grouper(key='duration',freq='5Min'))['started_at'].agg('count')
```

### .agg() 

Kan bruke hvis vi vil kjøre ulike aggegreringsfunksjon på ulike kolonner
```python
df_out = (df.groupby("key").
              agg({"col1":'mean', 'col2':'count'}).
              rename(columns={'col1':'avg_col1', 'col2':'num_col2'})
)
```

### Pivot tables

Lage en df fra eksisterende df for å analysere spesifikk problem. Bestemmer index. Lar verdiene i én av kolonnene være kolonner(kategorier) i ny df. Bestemmer hvilken kategori som skal være verdi for hver index langs nye kolonner. Spesifisere aggregeringsfunksjon for alle verdiene for gitt katagori for hver index (flere observasjoner inn i samme rute --> aggregering til ett tall).

eks: df.pivot_table(values=BNP , index=ÅR ,columns =LAND, agg=np.mean) 

## Reshape


Vi kan representerer de samme dataene på ulike måter. 

1. Long (alt er i index...)
2. Wide (alt er i kolonne...)

Vi har fire grunnoperasjon for å omforme data

1. stack (sender ting ned i index, flere levels på indexen)
2. unstack (sender ting opp i kolonne, mindre levels på indexen)
3. set_index
4. reset_index

Har operasjoner som bruker disse under the hood

1. melt (gjør long)
2. pivot (index, col, value) <- wide?
3. pivot_table, generalisering som gjør at vi kan spesifisere agg funksjon dersom flere verdier til (index, col) par

Eksempel på melt.. Hvis jeg har mange kolonner med verdier, eks: uke 1, uke 2, uke 3..... vil heller ha key value (en kolonne med uke, en kolonne med verdien den uke). Bare spesifiser at alle andre kolonner er id-vars

```python
df.melt(id_vars, # kombinasjon av kolonne som unik identifiserer obs
        value_vars, # bruker alle som ikke er i id_vars som default.. kan spesifisere for å droppe resten..
        var_name, # navn på kolonnen med verdier fra opprinnelige kolonner
        value_name) # navn på kolonnen med verdi som korresponedere med opprinelig kolonner
```

Kan ha tabell i stacked format. egen kolonne som hva slags type variabel og deretter kolonne med verdier,

|index|type |verdi |
|---|---|---|
|1|a|1|
|1|b|2|
|2|a|3|

for å få dette på tidy format der innehold i hver celle korresponderer med et (key,type) par så må jeg pivote

```python
df.pivot(index='index', # hver unik verdi i kolonnen utgjør én rekke i ny df
         columns='type', # hver unik verdi utgjør navn på én kolonne i ny df
         values='verdi', # hvilke kolonner som fyller verdiene i nye kolonner
        )
```

## Transformasjoner

### Map

Vil gjøre funksjon elementvis på rekke. Gir ut array med samme størrelse der output i rekken avhenger av input i rekken. 

Kan ta dict som argument hvis vi vil omkode verdier i henhold til tabell
```python
table = {'old_val0':'new_val0', 'old_val1':'new_val1'}
df['col'] = df['col'].map(table)
```
Kan også bruke lambda for å lage custom funksjoner
```python
df['col'] = df['col'].map(lambda x: 2*x) # x er verdi i rekken av array
```

### Apply

hm

#### Applymap

hmm

### pipe

Tingen er at .apply() anvender funksjon på hvert element av iterable, men har ikke helt bilde av helheten.

Hvis jeg i stedet bruke .pipe() på groupby så vet den hvor mange grupper det er og sånn.

hmmm

## Kombinere dataframes

Har fire typer joins som bestemmer hvilke observasjoner som blir med i det nye datasettet.
1. inner
2. left
3. right
4. outer

for de tre nederste blir observasjon som ikke har verdi i en kolonne paddet med nans

### concat

Hvis har samme kolonner eller samme index i to dataframes så er det enklest å bare slenge de sammen med

pd.concat([liste av dfs],axis=)

padder på med nans avhengig av hvilke type join. Beholder indexverdi.

### merge

Bruker verdi i felles kolonner til å merge verdier. Tenk at vi har observasjon om hvilke land individer kommer fra. På bakgrunn av dette vil vi koble på mer informasjon om landet på hvert individ. Vi har et annet datasett med informasjon om hvert land. Kan da koble dette på våre individdata med land som 'key' kolonne. Den driter i

pd.merge(left,right) tilsvarer left.merge(right)

left.merge(right,on='key',how=) 

- Ulike navn på kolonneindex som merger på: left_on=, right_on=
- Hvis kolonnen er plassert som index så bruker vi bare left_index=True, right_index=True. Hvis indexen har navn så kunne vi brukt left_on=

Hvis vi vil merge to datasett med samme type index (ie. liste av navn, land .. whatever) så bruker vi pd.merger(a,b,options). Siden det ofte vil være slik at noen index ikke har verdier for noen av kolonnene har vi masse options for hvordan merge.

1) how = left -- kun obs som har verdier til kol a

2) how = right -- tilsvarende

3) how = inner (snitt) -- verdier på begge

4) how = outer (union) -- verdier på minst én

Vi må også spesifisere hva som er indexene for mergingen. left_on, right_on, left_index=True etc|

### join

Convience funksjon som bruke pd.merge under the hood, men kan være bedre valg dersom vi bruker index til å koble data

### Legge til series (rad og kolonne)

Hvis vi bruker df.append() så lages nytt objekt. Må sende en series eller dataframe som argument. 

Eks: df = dt.append(pd.Series({dict: keys:values}), der keys korresponderer med kolonne indexer 


## Querying dataframe 

### Boolean mask

Kan bruke bools til å indexe. Gir ut array som "bli lagt oppå dataframen". Får ut de verdiene som korresponderer med True. 

Boolean masking er viktig!! Apply operatorer til dataframe.

1) Lage boolean array, f.eks: df[<column name>]<condition>. Får tilgang til kolonne og tester hver verdi opp mot condition. Returner array med samme størrelse der verdiene er enten true/false.

2) "Apply the mask to the dataframe". Lager nå df, df_new = df.where(<mask>). Altså bruke where method og boolsk array som input. Ny df har samme størrelse som gamle, men data fra rows hvor betingelse = false ---> NaN verdier. For å droppe disse rekke kan vi bruke df = df.dropna()
    

Kan gjøre begge deler i ett steg: df_new = df[df[<kolonne>]<condition>]] Bruke boolsk array som index til original df. Jævla clean og gjør enkelt å lage mer kompliserte logiske operasjon ved å binde sammen med | (eller) & (og). Der hver del er innrammet i parantes.

```python
df_new = df[(df[["gold"] > 0) & (df["gold1"] == 0])]
df_new = df[<col>][<bool mask>].
df_new = df[df.NavnPåKolonne == verdi]
```

### Query

Tar string som den evaluerer

### Where

Kan lage nye kolonne fra ifelse conditional med

```python
df['new_col'] = np.where(df['old_col']=='value', 'A', 'B')
```

## Dato-funksjonaliteter i pandas

Tidspunkt har mange egenskaper. De har rangering der større verdier er senere enn små verdier, men det er langt fra tilstrekkelig å representere denne ordinale rangeringen med tall. Det finnes mange ulike målenheter på avstand i tid (dager, sekunder, millisekunder, mm.) og det er ikke alltid trivielt å konvertere mellom disse, samt at det avhenger av tidssone. Dessuten er det mye informasjon assosiert med gitte tidspunkt (hvilken uke eller dag det er, mm).

Dersom vi gir informasjon om tidspunkt en riktig representasjon kan pandas gjøre mye arbeid for oss. Det gir oss tilgang til informasjon og det kan tolke hva vi mener slik at det blir enkelt å filtrere observasjoner ut fra årstall, ukedag eller lignende. Det kan også tolke avstand mellom tidspunkt i ulike måleenheter. For å oppnå dette må vi bruke classer som er *time aware*.

Av uvisse grunner har det mer intuitiv funksjonalitet når datetime array er indeks i stedet for kolonne i dataframe... Gir vel uansett stort sett mening å ha det som index

### Objekter

lage datetimeindex
- pd.date_range(start, period, freq)
- pd.period_range()

Timestamp objekt, har mange egenskaper vi kan få ut

Period, dag/måned/året... representer periode i stedet for snapshot. har begynnelse og slutt

Timedelta for offset (differanse i tid). Kan bruke dette i algebra med tidspunkt..

#### Datetime

Kan konstruere index som består av datetime med 
```python
date_index = pd.date_range(start, # string som kan bli parset som dato..
                           period, # int med antall perioder
                           freq, # avstand mellom perioder, 'D' for daily, 'H' hourly, ..
                           end, # kan alternativt spesifisere sluttdato
                           tz # kan spesifisere timezone. Kan deretter konverte med .tz_convert('time zone')
                           )
```

Annen måte å konstruere er .to_datetime() fra eksisterende series med strings eller lignende

```python
pd.to_datetime(array, # prøver å parse innhold i array, men vi må ofte i praksis hjelpe litt til med flere argument
               format, # spesifisere format, litt usikker på hvordan... eks '%Y%m%d'
               day_first, # alternativt kan vi gi litt tips
               )
```

#### Period

Kan konstruere index av periods på måte som er analog til date_range med `pd.period_range()`

representerer intervall av tidspunkt. har begynnelse og slutt. tror ulike datetime/timestamp kan være inni gitt Period. 

#### Timestamp

vet forøvrig ikke hva som er forskjellen på timestamp og datetime

#### Timedelta

differanse mellom datetime objekt, greit når vi skal gjøre aritmetikk.. lengde tid (duration) noe tar

Ikke helt trivielt å formatere string output (imotsetning til datetime objekt). Må bruke
```python
components = df['time_delta_col'].dt.components
```

### datetime interface

Bruker .dt til å få interface til egenskaper for datetime objekt

Indikere om det er helg,

```python
df['weekend'] = np.where(df['date'].dt.dayofweek < 5, 0, 1)
```

### Vinduer for aggregering

#### Rolling window

Litt analog til groupby, men i stedet for å konstruere egne dataframes filtrert for spesifikke verdier av variabler i angitt kolonner så vil det for hvert tidspunkt være en tilhørende dataframe med verdiene fra k tidspunkt foran.

Vi kan angi aggregeringsfunksjon på disse dataframene slik at vi får én verdi tilhørende hvert tidspunkt, f.eks gjennomsnitt og varians for å undersøke stasjonaritet, eller f.eks. max/min verdi av tilhørende dataframe. Merk at dette vil gi NA for tidspunkt som ikke har nok foregående verdier til å fylle ut k tidligere tidspunkt.

Den bruker bare tidligere data. Det betyr at f.eks. gjennomsnitt alltid er litt på etterskudd. Finnes alternative tilnærminger som bruker data på begge sider av det angitte tidspunktet... litt usikker på fordel og ulemper..

```python
r = df.rolling(window, # antall tidspunkt. Flere tidspunkt gir smoothere plot. 
               min_periods, # kan spesifisere minste antall tidligere periode for å ikke gi NA
               center # kan velge å plassere verdi til senter av vindu...
               )

r['navn på col'].agg_func() # gir ut dataframe som vi kan plotte
```

#### Expanding

Bruker info fra alle observasjoner før tidspunkt, i motsetning til rolling vindu som har en angitt størrelse. Blir litt sånn som cumulative sum og sånn...

### Lags

Kan lage lagged kolonner med .shift()

## String methods

Hvis kolonne er string kan jeg bruke df.col.str for å få tilgang til metoder. Dette er ofte bedre å raskere enn å gjøre en eksplisitt loop og bruke string metode på innholdet i hver celle. Har de fleste (alle?) metodene som native python har på strings. Har i tillegg noen egne funskjonaliteter knyttet til pandas, feks:
```python
# lage dummmies der hver observasjon kan inneholde flere verdier i form av string
df['col'].str.get_dummies(sep='|') 

# Hvis jeg vil bruke flere str operasjoner i chain må jeg bruke .str etter hver, eks
data['Min_Salary']=data['Min_Salary'].str.strip(' ').str.lstrip('$').str.rstrip('K').fillna(0).astype('int')
```


## Diverse data cleaning 

### Eksempler

```python
# Omdefinere verdier i kolonne i henhold til tabell/dict
table = {'Amer-Indian-Eskimo':'Native','Asian-Pac-Islander':'Pacific'}
df['race'] = (df['race'].
    str.strip().
    astype('category').
    cat.rename_categories(table))
```

```python 
# Vil ha kolonnenavn i camel_case i stedet for SnakeCase
def camel_to_snake(s):
    return ''.join('_'+ch.to_lower() if ch.isupper() else ch for ch in s)
df.columns = [camel_to_snake(column) for column in df.columns]

```

```python
# fjerne alle symboler i string som ikke er numerisk
df['name'] = df['name'].map(lambda x: ''.join([ch for ch in x if ch.isdigit()]))
```

## Generelle tips

Hvis vi jobber med stor dataframe og kun er interessert i liten del for å svare på et gitt spørsmål, så kan det være en god idé å lage en ny dataframe som er subset av den opprinnelige. 

Stiltips:

1) Unngå ][ , chain indexing

2) Chain metoder. En måte å gjøre dette mer oversiktelig er å fordele det over flere linjer (analog til dplyr), eks:

```python
(df.groupby["NN"]
    .MM # velge kolonne
    .dropna() # kommentar
    .mean() # kommentar
    )
```

## Annet

### pd.cut()

Når vi vil dele kontinuerlige data inn i bins. Konstruere 'factor' med levels, kan deretter lage dummies

```python
pd.cut(array,
       bins, # Antall bins (lager slik at uniform antall i hver) eller konstruere med lik avstand mellom
       retbins=True, # spesifisere om returnere cutoffs til bins
       labels # kan bruke np.range(len(bins)) hvis kun interessert i relativ plassering
       )
```

bins kan ta ulike form:
1. int, spesifiserer antall bins, partisjonerer (min(x),max(x)) inn i like store intervall
2. sekvens av tall (a,b,c,..), lager intervall (a,b],(b,c] hvis right=True. Gir NaN hvis ikke i angitte intervall.
3. IntervalIndex (vet ikke)


### Andre typer objekt i pandas

Vi kjenner til groupby-objekt. Finnes også andre objekt som vi kan utføre aggregeringsfunksjoner på og få ut Series eller df.

-  a = pd.Series(..), a.rolling(window=antall obs) <- gir oss et rolling objekt med en rekke metoder.
-  a.mean() gir oss en Series med rullende gjennomsnitt. Denne seriesen kan vi f.eks. bruke til å plotte.
-  Et alternativ til aritmetisk gjennomsnitt er at tyngden til observasjon er vektet etter tid. Kan bruke a.ewm() til å få eksponentielt vektet ting.

### Nyttige metoder

-  a.pct_change() gir oss en ny series (eller df hvis flere kolonner) der hver observasjon fra i=1 -> n er erstattet med df[i]/df[i-1]-1. Husk å gange med 100 for å få prosentvis endring
- a.nlargest() gir series med index og verdi til n største verdiene i en series
- a.transform(func) gir en series med samme størrelse der vi har kjørt en funksjon på den.. Kan også brukes på dataframes. Uansett, eksempel er b = a.transform(lambda x: pd.cut(x, 100))
- a.str gir oss tilgang til mange string metoder som vi kan anvende på alle elementer i series. raskere enn å kjøre det i en loop.
- df.iterrows() gir oss et generatorobjekt som vi kan bruke til å loope oss gjennom df row by row. Hvert element er en tuple med rowindex og rowen som series. Følger da at index til min row series er kolonnenanvnene. 
- agg og apply, vet ikke helt hva som er forskjellen
- stack gjøre kolonnenavn til indexverdier
- unstack gjør indexverdier til kolonnenavn
- reindex til å velge subset av index/kolonne når jeg har liste av tall jeg vil subsette med
- idxmax() gir første index til høyeste verdi langs gitt index.. hvis axis=1 så får vi kolonnen med høyest verdi 

### Nyttige funksjoner

- ufuncs fungerer også på dataframe
- Har innebygde funksjoner som gir én output per kolonne/rekke (typ statistiske/matematiske funksjoner)
- pd.get_dummies() lager dummy dataframe fra kategorisk variabel. Kan spesifisere predix="string" slik at de får kolonnenavn ["string_0","string_1,...]
- Kan også bruke det dersom vi bare vil ha én kolonne, df['col'] = pd.get_dummies(df.col)[val], der val er verdi av kol som vi vil skal ta verdi 1 (resten 0)

## Annet

Det er ganske greit å rekonstruere en kategorisk variabel fra dummies (f.eks for plotting purposes eller hvis jeg vil kjøre groupby på det). Kategorisk er egentlig alltid en bedre representasjon før vi kjører det inn i algoritme, så synes egentlig ikke jeg bør se noen fuckings dummies okay det dritet der bør det være mulig å skjule.

```python
subset_cols = [col for col in df.columns if col.startswidth('prefix')]
subset_df = df[subset_cols]
new_col = subset_df.idxmax(axis=1) # får nye series der tar kolonnenavn (fra kol der val=1) som verdi
```